# Animatlab to MuJoCo Convert (WIP)

As of now, this converter does not work

## TO-DOS:
* [ ] Add Muscles and motors
* [ ] Understand and fix the angle issue
* [ ] Add global variables automatically
* [ ] Add mass + density
* [ ] Add colors

## Imports
For this project I'm using xmltodict to convert the .asim file into pythonian and ElementTree to generate the new xml for MuJoCo.

In [65]:
import xmltodict
import xml.etree.ElementTree as et
from scipy.spatial.transform import Rotation as R
import numpy as np
import math


## Step 1: Import the Test File

I'm testing with the rat hindlimb model which we're using for testing our SNS

In [66]:
with open("xmls/Whole_leg_sim_Standalone.asim",'r') as f:
    dicto = xmltodict.parse(f.read())
rat = dicto['Simulation']['Environment']["Organisms"]["Organism"]

[Link for this helper method](https://stackoverflow.com/questions/58997792/how-do-you-convert-euler-angles-to-the-axis-angle-representation-in-python)

In [67]:

def euler_xyz_to_axis_angle(x_e, y_e, z_e, normalize=True):
    # Assuming the angles are in radians.
    c2 = math.cos(x_e/2)
    s2 = math.sin(x_e/2)
    c3 = math.cos(y_e/2)
    s3 = math.sin(y_e/2)
    c1 = math.cos(z_e/2)
    s1 = math.sin(z_e/2)
    c1c2 = c1*c2
    s1s2 = s1*s2
    w = c1c2*c3 - s1s2*s3
    x = c1c2*s3 + s1s2*c3
    y = s1*c2*c3 + c1*s2*s3
    z = c1*s2*c3 - s1*c2*s3
    angle = 2 * math.acos(w)
    if normalize:
        norm = x*x+y*y+z*z
        if norm < 0.001:
            # when all euler angles are zero angle =0 so
            # we can set axis to anything to avoid divide by zero
            x = 1
            y = 0
            z = 0
        else:
            norm = math.sqrt(norm)
            x /= norm
            y /= norm
            z /= norm
    return x, y, z, angle

def lst2str(lst):
    oot = ""
    for l in lst:
        oot = oot + " " + str(l)
    return oot[1:]

In [76]:
def process_rb(rb):
    new_rb = {}
    keys = ["Name","ID","Mass","Density","Position","Rotation","Type",
            "IsCollisonObject","IsContactSensor",
            "StimulusTension","Length Tension","MaximumTension","Kse","Kpe","B","Attachments"]
    for key,value in rb.items():
        if key in keys:
            new_rb[key] = process_item(key,value)
    
    if "ChildBodies" in rb.keys():
        new_rb["ChildBodies"] = [process_rb(child) for child in rb["ChildBodies"]["RigidBody"]]
    if "Width" in rb.keys(): #if width exists, assume the rest do...
        new_rb["size"] = "{} {} {}".format(float(rb["Length"])*50,float(rb["Height"])*50,float(rb["Width"])*50)
    if "Joint" in rb.keys():
        if isinstance(rb["Joint"],list):
            new_rb["Joints"] = [process_joint(child) for child in rb["Joint"]]
        else:
            new_rb["Joint"] = process_joint(rb["Joint"])
    
    return new_rb

def process_joint(joint):
    new_jnt = {}
    keys = ["Name","Position","Rotation","Type","Size",
            "MaxForce","MaxVelocity","EnableMotor","MotorType","ServoGain"]
    relaxations = []
    new_jnt["limited"] = False
    for key,value in joint.items():
        if key in keys:
            new_jnt[key] = process_item(key,value,is_jnt=True)
        if key in ["LowerLimit","UpperLimit"]:
            new_jnt[key] = {}
            new_jnt[key]["LimitPos"] = float(value["LimitPos"])
            new_jnt[key]["Damping"] = float(value["Damping"])
            new_jnt[key]["Restitution"] = float(value["Restitution"])
            new_jnt[key]["Stiffness"] = float(value["Stiffness"])
            new_jnt["limited"] = True
        if key.__contains__("Relaxation") and value["Name"].__contains__("Rotation"):
            relaxations.append(value)
        if key == "Friction":
            if value["Enabled"] == True:
                new_jnt["Friction"] = {}
                new_jnt["Friction"]["Type"] = float(value["Type"])
                new_jnt["Friction"]["Coefficient"] = float(value["Coefficient"])
                new_jnt["Friction"]["MaxForce"] = float(value["MaxForce"])
                new_jnt["Friction"]["Loss"] = float(value["Loss"])
    
    rot_axis = [0,0,0]
    for r in relaxations:
        if r["Name"].__contains__("X"):
            rot_axis[0] = True
        if r["Name"].__contains__("Y"):
            rot_axis[1] = True
        if r["Name"].__contains__("Z"):
            rot_axis[2] = True
    new_jnt["axis"] = "{:n} {:n} {:n}".format(not rot_axis[0],not rot_axis[1],not rot_axis[2])
    return new_jnt

def process_item(key,param,is_jnt=False):
    if key =="Position":
        return "{} {} {}".format(float(param["@x"])*100,float(param["@y"])*100,float(param["@z"])*100)
    elif key =="Rotation":
        x0 = float(param["@x"])
        y0 = float(param["@y"])
        z0 = float(param["@z"])
        
        if is_jnt:
            r = R.from_euler("xyz",[x0,-z0,y0],degrees=True)
            r_mat = r.as_matrix()
            [x, y, z] =  np.matmul(r_mat,np.array([1, 0, 0]).transpose())
            print(x,y,z)
            return "{} {} {}".format(x,y,z)
        else:
            return "{} {} {}".format(x0,y0,z0)
    else:
        try:
            float(param)
            return float(param)
        except:
            return param

In [77]:

dic = process_rb(rat["RigidBody"])
global_rot = dic["Rotation"].split(" ")
global_rot[0] = str(float(global_rot[0]) + math.pi / 2)
dic["Rotation"] = " ".join(global_rot)
#for i in rat["RigidBody"]["ChildBodies"]["RigidBody"]:
#    if "Joint" in i.keys():
#        print(i["Joint"])


0.9996241194093897 0.02741568702407634 0.0
0.9996241194093896 0.0 0.02741568702407634
0.9996241194093897 0.02741568702407634 0.0
0.9996241194093897 0.02741568702407634 0.0
0.9996241194093896 0.0 0.02741568702407634
0.9996241194093897 0.02741568702407634 0.0


In [78]:
#[body["Name"] for body in dic["ChildBodies"][5]["ChildBodies"]]
item = dic
print(item["Name"])
print(item["Rotation"])
#print(item["Rotation"])
#print(item["Attachments"])


Pelvis
3.1417963267948963 0.5236 -3.142


## Step 2: Make the MuJoCo XML file

In [85]:
def xmlify_body(dic,attach_ids={},root=0):
    #print(dic["Name"])
    #print(dic["Type"])
    muscles = []
    body = et.Element("body",attrib={
        "name":dic["Name"],
        "pos":dic["Position"],
        "euler":dic["Rotation"]
    })
    body.append(et.Element("geom",attrib={
        "type": "box",
        "size": dic["size"]
    }))
    if "ChildBodies" in dic.keys():
        for value in dic["ChildBodies"]:
            if value["Type"] == "Attachment":
                body.append(et.Element("site",attrib={
                    "name":value["Name"],
                    "pos":value["Position"]
                }))
                attach_ids[value["ID"]] = value["Name"]
            if value["Type"].__contains__("Muscle"):
                muscles.append(value)
            if value["Type"] == "Box":
                child, child_muscles, _ = xmlify_body(value,attach_ids=attach_ids,root=1)
                body.append(child)
                muscles.append(child_muscles)
    if "Joint" in dic.keys():
            joints = []
            if isinstance(dic["Joint"],list):
                joints = dic["Joint"]
            else:
                joints.append(dic["Joint"])
            for value in joints:
                if value["limited"]:
                    body.append(et.Element("joint",attrib={
                            "name":value["Name"],
                            "pos":value["Position"],
                            "axis":value["axis"],
                            "limited":"true",
                            "range":"{} {}".format(value["LowerLimit"]["LimitPos"],value["UpperLimit"]["LimitPos"]),
                            "stiffness":str(value["LowerLimit"]["Stiffness"]),
                            "damping":str(value["LowerLimit"]["Damping"])
                        }))
                else:
                    body.append(et.Element("joint",attrib={
                        "name":value["Name"],
                        "pos":value["Position"],
                        "axis":value["axis"]
                    }))
                

    if root == 0:
        pass
        #for muscle in muscles:
    return body, muscles, attach_ids

In [86]:
root = et.Element("mujoco")
root.append(et.Element("compiler",attrib={"coordinate":"local","angle":"radian","eulerseq":"xyz"}))

default = et.SubElement(root,"default")
default.append(et.Element("geom",attrib={"rgba":".8 .4 .6 1"}))

asset = et.SubElement(root,"asset")
asset.append(et.Element("texture",attrib={"type":"skybox",
                                          "builtin":"gradient",
                                          "rgb1":"1 1 1","rgb2":".6 .8 1",
                                          "width":"256",
                                          "height":"256"
                                         }))
world_body = et.SubElement(root,"worldbody")
world_body.append(et.Element("geom",attrib={"name":"floor",
                                            "pos":"0 0 -10",
                                            "size":"10 10 0.125",
                                            "type":"plane",
                                            "condim":"3",
                                            "rgba":"1 1 1 1"
                                           }))
world_body.append(et.Element("light",attrib={"pos":"0 15 15",
                                            "dir":"0 -1 -1",
                                            "diffuse":"1 1 1"
}))
body,muscles,attach_ids = xmlify_body(dic)
world_body.append(body)
with open("xmls/NewRat.xml","wb") as fp: 
    et.indent(root)
    tree = et.ElementTree(root)
    tree.write(fp)

In [73]:
import json
with open('outfile.json','w') as fp:
    json.dump(dic,fp)